# Chapter 2: Graphs 

## Think Complexity Second Edition by Alan Downey

In [ ]:
import networkx as nx
import numpy as np

import plotly_express as px

COLORS = px.colors.qualitative.Alphabet

In [ ]:


G = nx.Graph()

positions = dict(Albany=(-74, 43),
                 Boston=(-71, 42),
                 NYC=(-74, 41),
                 Philly=(-75, 40))

G.add_nodes_from(positions)


drive_times = {('Albany', 'Boston'): 3,
               ('Albany', 'NYC'): 4,
               ('Boston', 'NYC'): 4,
               ('NYC', 'Philly'): 2}

G.add_edges_from(drive_times)


nx.draw(G, positions, node_color=np.random.choice(COLORS), node_shape='*', node_size=2500, with_labels=True)
nx.draw_networkx_edge_labels(G, positions, edge_labels=drive_times)

In [ ]:
positions['Montreal'] = (-73.6, 45.5)
drive_times[('Montreal', 'NYC')] = 6 

G.update(nodes=positions, edges=drive_times)

nx.draw(G, positions, node_color=np.random.choice(COLORS), node_shape='*', node_size=2500, with_labels=True)
nx.draw_networkx_edge_labels(G, positions, edge_labels=drive_times)

In [ ]:
def all_pairs(nodes):
    for i, u in enumerate(nodes):
        for j, v in enumerate(nodes):
            if i < j:
                yield u, v
                
        
list(all_pairs(range(4)))

def make_complete_graph(n):
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from(all_pairs(nodes))
    return G

Gc = make_complete_graph(5)
list(Gc.edges)

In [ ]:
complete = make_complete_graph(10)
complete.number_of_nodes()

len(complete.edges)

In [ ]:
def all_directed_pairs(nodes):
    for i, u in enumerate(nodes):
        for j, v in enumerate(nodes):
            if i != j:
                yield u, v
                
def make_complete_digraph(n):
    G = nx.DiGraph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    
    edges = all_directed_pairs(nodes)
    
    G.add_edges_from(edges)
    
    return G

In [ ]:
directed_complete = make_complete_digraph(5)
len(directed_complete.edges)

nx.draw(directed_complete, with_labels=True)

In [ ]:
def flip(p):
    return np.random.random() < p

def random_pairs(nodes, p):
    for edge in all_pairs(nodes):
        if flip(p):
            yield edge

In [ ]:
def make_random_graph(n, p):
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from(random_pairs(nodes, p))
    return G

In [ ]:
np.random.seed(100)
random_graph = make_random_graph(10, 0.2)
len(random_graph.edges())

In [ ]:
nx.draw_circular(random_graph, 
                 node_color=np.random.choice(COLORS), 
                 node_size=1000, 
                 with_labels=True)

In [ ]:
def reachable_nodes(G, start):
    seen = []
    stack = [start]
    
    while len(stack) > 0:
        node = stack.pop()
        neighbors = G.neighbors(node)
        for n in neighbors:
            if n not in seen:
                seen.append(n)
                stack.append(n)
                
    return seen

reachable_nodes(G, 'Montreal')

In [ ]:
def reachable_nodes_book(G, start):
    seen = set()
    stack = [start]
    while stack:
        node = stack.pop()
        if node not in seen:
            seen.add(node)
            stack.extend(G.neighbors(node))
    return seen

reachable_nodes_book(G, 'Montreal')

In [ ]:
reachable_nodes(complete, 0)
reachable_nodes_book(complete, 0)

In [ ]:
reachable_nodes(random_graph, 1)
reachable_nodes_book(random_graph, 1)

In [ ]:
for n, p in zip([100, 200, 300], [0.1, 0.2, 0.5]):
    graph = make_random_graph(n, p)
    assert set(reachable_nodes(graph, 0)) == reachable_nodes_book(graph, 0)

In [ ]:
np.random.seed(40)
test_random = make_random_graph(1000, 0.15)

In [ ]:
%%timeit
reachable_nodes(test_random, 0)

In [ ]:
%%timeit 
reachable_nodes_book(test_random, 0)

In [ ]:
def is_connected(G):
    return len(G.nodes) == len(reachable_nodes_book(G, list(G.nodes.keys())[0]))

is_connected(complete)

In [ ]:
is_connected(test_random)

In [ ]:
low_p_random = make_random_graph(8, 0.1)
is_connected(low_p_random)

In [ ]:
nx.draw(low_p_random, node_color=np.random.choice(COLORS))

In [ ]:
def reachable_nodes_directed(G, start):
    seen = set()
    stack = [start]
    
    while stack:
        node = stack.pop()
        if node not in seen:
            seen.add(node)
            stack.extend(G.successors(node))
            
    return seen

def is_connected_directed(G):

    return len(G.nodes) == len(reachable_nodes_directed(G, list(G.nodes.keys())[0]))

In [ ]:
is_connected_directed(make_complete_digraph(10))

In [ ]:
not_connected_digraph = nx.DiGraph()
not_connected_digraph.add_nodes_from([0, 1, 2, 3])
not_connected_digraph.add_edges_from([(0, 1), (1, 2), (3, 0)])

nx.draw(not_connected_digraph)

In [ ]:
is_connected_directed(not_connected_digraph)

In [ ]:
def prob_connected(n, p, iters):
    return np.mean([is_connected(make_random_graph(n, p)) for _ in range(iters)])

for n, p in zip([10, 15, 20], [0.1, 0.15, 0.2]):
    print(f'The probability of a graph being connected with {n} nodes and {p} probability two nodes are connected is {prob_connected(n, p, 100):.2f}%.')

In [ ]:
def pstar(n):
    return np.log(n) / n

pstar(10)

In [ ]:
ns = np.arange(2, 25)
ps = np.linspace(0.1, 0.61, num=50)

from itertools import product
from collections import defaultdict
results = []

for n, p in product(ns, ps):
    results.append((n, p, prob_connected(n, p, 100)))

In [ ]:
import pandas as pd
import cufflinks as cf

probs = pd.DataFrame(results, columns=['n', 'edge_prob', 'connected_prob'])

pstar_comparison = probs.groupby('n').apply(lambda x: x.loc[x['connected_prob'] > 0.5, 'edge_prob'].min()).to_frame().rename(columns={0:'empirical'})
pstar_comparison['theoretical'] = pstar(pstar_comparison.index)
pstar_comparison

In [ ]:
pstar_comparison.iplot(kind='bar', xTitle='Number of Nodes', yTitle='PStar', title='Value of Probability of Node Connections at which More than 50% of Randomly Drawn Graphs are Connected')

In [ ]:
probs.head()

In [ ]:
probs['nodes'] = [f'{n}_nodes' for n in probs['n']]
probs.iplot(
    mode='lines',
    x="edge_prob",
    y="connected_prob",
    categories="nodes",
    title="Connected vs Edge Prob by Number of Nodes",
)

In [ ]:
fig = px.line(
    probs,
    x="edge_prob",
    y="connected_prob",
    color="nodes",
    title="Connected vs Edge Probability by Number of Nodes",
)
fig

In [ ]:
import random
random.sample([(1, 0), (2, 3)], 1)

In [ ]:
def make_random_graph_alt(n, m):
    nodes = range(n)
    all_edges = all_pairs(nodes)

    if m > ((n**2) - n) / 2:
        raise ValueError('More edges than is possible!')
    
    selected_edges = random.sample(list(all_edges), m)
    
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(selected_edges)
    
    return G

random_G = make_random_graph_alt(10, 40)
nx.draw(random_G, with_labels=True)

In [ ]:
def prob_connected_alt(n, m, iters):
    return np.mean([is_connected(make_random_graph_alt(n, m)) for _ in range(iters)])

In [ ]:
ns = np.arange(2, 21)
ps = np.linspace(0.1, 0.61, num=50)

results = []

for n, p in product(ns, ps):
    m = int( p * ( (n ** 2) - n) / 2)
    
    
    prob_connected_graph = prob_connected_alt(n, m, 100)
    
    results.append((n, p, m, prob_connected_graph))
    
results = pd.DataFrame(results, columns=['n_nodes', 'prob_edge', 
                                        'n_edges', 'prob_connected'])
results.head()
    

In [ ]:
pstar_comparison = results.groupby('n_nodes').apply(lambda x: x.loc[x['prob_connected'] > 0.5, 'prob_edge'].min()).to_frame().rename(columns={0: 'empirical'})
pstar_comparison['theoretical'] = pstar(pstar_comparison.index)
pstar_comparison

In [ ]:
pstar_comparison.iplot(kind='bar', xTitle='Number of Nodes', yTitle='Pstar', title='Probablity of Edge between Nodes at which Probability of Connected Graph is Greater than 50%')

In [ ]:
px.line(results, x='prob_edge', y='prob_connected', color='n_nodes', title='Probability of Connected Graph vs Probability of Edge between Nodes')